### Section 1: Setup

1.1: Install dependencies

In [ ]:
# ADK in Python
!pip install google-adk

  Using cached anyio-4.11.0-py3-none-any.whl.metadata (4.1 kB)
  Using cached graphviz-0.21-py3-none-any.whl.metadata (12 kB)
  Using cached python_dotenv-1.2.1-py3-none-any.whl.metadata (25 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached idna-3.11-py3-none-any.whl.metadata (8.4 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached proto_plus-1.26.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached pyasn1_modules-0.4.2-py3-none-any.whl.metadata (3.5 kB)
  Using cached rsa-4.9.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached sqlparse-0.5.3-py3-none-any.whl.metadata (3.9 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached pyparsing-3.2.5-py3-none-any.whl.metadata (5.0 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached attrs-25.4.0-py3-

In [2]:
# Read environment variables from .env file

!pip install python-dotenv

1.2: Configure your Gemini API Key

In [ ]:
import os
from dotenv import load_dotenv

# Load .env file
load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if not GOOGLE_API_KEY:
    raise ValueError("GOOGLE_API_KEY tidak ditemukan. Pastikan sudah menaruhnya di file .env")

print("Google API key loaded!")


✅ Google API key loaded!


1.3: Import ADK components

In [7]:
import os
from dotenv import load_dotenv

from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.genai import types

print("ADK components imported successfully.")

ADK components imported successfully.


In [9]:
# Load API key
load_dotenv()

# Setup environment
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

1.5: Configure Retry Options

When working with LLMs, you may encounter transient errors like rate limits or temporary service unavailability. Retry options automatically handle these failures by retrying the request with exponential backoff.

In [8]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

### Section 2: Your first AI Agent with ADK¶

2.1 What is an AI Agent?
You've probably used an LLM like Gemini before, where you give it a prompt and it gives you a text response.

Prompt -> LLM -> Text

An AI Agent takes this one step further. An agent can think, take actions, and observe the results of those actions to give you a better answer.

Prompt -> Agent -> Thought -> Action -> Observation -> Final Answer

In this notebook, we'll build an agent that can take the action of searching Google. Let's see the difference!

2.2 Define your agent¶

These are the main properties we'll set:

- name and description: A simple name and description to identify our agent.
- model: The specific LLM that will power the agent's reasoning. We'll use "gemini-2.5-flash-lite".
- instruction: The agent's guiding prompt. This tells the agent what its goal is and how to behave.
- tools: A list of tools that the agent can use. To start, we'll give it the google_search tool, which lets it find up-to-date information online.

In [10]:
root_agent = Agent(
    name="travel_itenerary_agent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="A simple agent that can answer general questions.",
    instruction="You are NaviTrip, a friendly and organized travel-planner AI agent. " \
    "You create detailed, realistic travel itineraries using efficient routing, " \
    "estimating costs, and adapting to the user's preferences and constraints. " \
    "You always ask clarifying questions if needed, but if enough data is provided, " \
    "you generate a full itinerary without asking again." \
    "Your responses should be in Indonesian language.",
    tools=[google_search],
)

print("Root Agent defined.")

Root Agent defined.


2.3 Run your agent

Now it's time to bring your agent to life and send it a query. To do this, you need a Runner, which is the central component within ADK that acts as the orchestrator. It manages the conversation, sends our messages to the agent, and handles its responses.

a. Create an InMemoryRunner and tell it to use our root_agent:

In [11]:
runner = InMemoryRunner(agent=root_agent)

print("Runner created.")

Runner created.


b. Now you can call the .run_debug() method to send our prompt and get an answer.

👉 This method abstracts the process of session creation and maintenance and is used in prototyping. We'll explore

In [14]:
response = await runner.run_debug("Buatkan itinerary 3 hari di Bali untuk pecinta pantai.")
print(response)


 ### Created new session: debug_session_id

User > Buatkan itinerary 3 hari di Bali untuk pecinta pantai.
travel_itenerary_agent > Tentu, ini adalah draf itinerary 3 hari di Bali yang berfokus pada pecinta pantai, lengkap dengan perkiraan biaya.

**Asumsi:**

*   Anda tiba di Bali pada Hari 1 pagi dan berangkat pada Hari 3 sore/malam.
*   Perkiraan biaya adalah untuk 1 orang, belum termasuk tiket pesawat internasional/domestik.
*   Akomodasi dan transportasi adalah pilihan standar (bukan mewah).

---

**Itinerary 3 Hari di Bali untuk Pecinta Pantai**

**Hari 1: Keindahan Selatan Bali - Kuta, Seminyak, dan Sunset di Uluwatu**

*   **Pagi (08:00 - 11:00):** Tiba di Bandara Internasional Ngurah Rai (DPS), transfer ke akomodasi di area Kuta/Seminyak. Check-in dan titip barang.
    *   **Aktivitas:** Jelajahi Pantai Kuta, nikmati pasir putihnya, dan rasakan ombak yang cocok untuk pemula.
    *   **Perkiraan Biaya:** Transportasi bandara ke hotel (Rp 100.000 - Rp 150.000), akomodasi (mulai 

**Asumsi:**

*   Anda tiba di Bali pada Hari 1 pagi dan berangkat pada Hari 3 sore/malam.
*   Perkiraan biaya adalah untuk 1 orang, belum termasuk tiket pesawat internasional/domestik.
*   Akomodasi dan transportasi adalah pilihan standar (bukan mewah).

---

**Itinerary 3 Hari di Bali untuk Pecinta Pantai**

**Hari 1: Keindahan Selatan Bali - Kuta, Seminyak, dan Sunset di Uluwatu**

*   **Pagi (08:00 - 11:00):** Tiba di Bandara Internasional Ngurah Rai (DPS), transfer ke akomodasi di area Kuta/Seminyak. Check-in dan titip barang.
    *   **Aktivitas:** Jelajahi Pantai Kuta, nikmati pasir putihnya, dan rasakan ombak yang cocok untuk pemula.
    *   **Perkiraan Biaya:** Transportasi bandara ke hotel (Rp 100.000 - Rp 150.000), akomodasi (mulai dari Rp 250.000/malam).
*   **Siang (11:00 - 14:00):** Makan siang di salah satu kafe tepi pantai di Kuta atau Seminyak.
    *   **Aktivitas:** Bersantai di Pantai Seminyak, nikmati suasana yang lebih tenang dibandingkan Kuta, atau coba pelajaran selancar singkat.
    *   **Perkiraan Biaya:** Makan siang (Rp 100.000 - Rp 150.000), sewa papan selancar/pelajaran singkat (Rp 100.000 - Rp 200.000).
*   **Sore (14:00 - 18:00):** Perjalanan menuju Uluwatu.
    *   **Aktivitas:** Kunjungi Pura Uluwatu yang ikonik, nikmati pemandangan tebing yang dramatis dan lautan biru. Saksikan pertunjukan Tari Kecak saat matahari terbenam (biasanya mulai pukul 18:00).
    *   **Perkiraan Biaya:** Transportasi Kuta/Seminyak ke Uluwatu (Rp 150.000 - Rp 200.000), tiket masuk Pura Uluwatu (Rp 50.000), tiket Tari Kecak (Rp 150.000).
*   **Malam (18:00 - Selesai):** Makan malam seafood segar di Jimbaran Bay sambil menikmati suara ombak.
    *   **Perkiraan Biaya:** Makan malam seafood (Rp 200.000 - Rp 300.000).
*   **Perkiraan Biaya Total Hari 1:** Rp 950.000 - Rp 1.450.000 (tidak termasuk akomodasi jika check-in di hari yang sama).

**Hari 2: Pesona Nusa Penida - Pantai Eksotis dan Keindahan Bawah Laut**

*   **Pagi (07:00 - 10:00):** Perjalanan ke Pelabuhan Sanur untuk menyeberang ke Nusa Penida.
    *   **Aktivitas:** Naik *fast boat* ke Nusa Penida. Sesampainya di sana, sewa motor atau mobil dengan sopir untuk berkeliling.
    *   **Perkiraan Biaya:** Transportasi ke Pelabuhan Sanur (Rp 50.000 - Rp 100.000), tiket *fast boat* PP Sanur-Nusa Penida (Rp 350.000 - Rp 500.000), sewa motor (Rp 75.000/hari) atau mobil dengan sopir (Rp 600.000 - Rp 700.000/hari).
*   **Siang (10:00 - 17:00):** Eksplorasi pantai-pantai ikonik di Nusa Penida.
    *   **Aktivitas:** Kunjungi Kelingking Beach (dengan pemandangan tebing berbentuk dinosaurus), Angel's Billabong (kolam alami), dan Broken Beach. Jika waktu memungkinkan, kunjungi Crystal Bay untuk *snorkeling*.
    *   **Perkiraan Biaya:** Tiket masuk objek wisata (Rp 10.000 - Rp 20.000 per lokasi), biaya parkir (Rp 5.000 - Rp 10.000), makan siang di warung lokal (Rp 70.000 - Rp 100.000).
*   **Sore (17:00 - 19:00):** Kembali ke Pelabuhan Sanur.
*   **Malam (19:00 - Selesai):** Kembali ke akomodasi di Bali. Makan malam santai di area akomodasi.
    *   **Perkiraan Biaya:** Makan malam (Rp 100.000 - Rp 150.000).
*   **Perkiraan Biaya Total Hari 2:** Rp 710.000 - Rp 1.080.000 (jika menyewa motor). Jika menyewa mobil dengan sopir, tambahkan sekitar Rp 500.000.

**Hari 3: Pantai Tersembunyi dan Keberangkatan**

*   **Pagi (08:00 - 12:00):** Pagi ini, jelajahi pantai yang sedikit berbeda.
    *   **Pilihan 1 (Nusa Lembongan/Ceningan):** Naik *public boat* atau *fast boat* sebentar ke Nusa Lembongan/Ceningan (jika tidak sempat di Hari 2). Nikmati Pantai Dream Beach, Mushroom Bay, atau Devil's Tear.
    *   **Pilihan 2 (Pantai di Bali Daratan):** Kunjungi Pantai Pandawa atau Melasti Beach di daerah Ungasan. Pantai-pantai ini menawarkan keindahan pasir putih dan air jernih dengan akses yang lebih mudah.
    *   **Perkiraan Biaya:** Transportasi ke pelabuhan (jika ke Lembongan) atau transportasi ke pantai (Rp 100.000 - Rp 150.000), tiket kapal (jika ke Lembongan, sekitar Rp 150.000 - Rp 200.000 PP). Biaya masuk pantai biasanya gratis atau sangat terjangkau.
*   **Siang (12:00 - 14:00):** Makan siang di kafe atau warung dekat pantai pilihan Anda.
    *   **Perkiraan Biaya:** Makan siang (Rp 100.000 - Rp 150.000).
*   **Sore (14:00 - Selesai):** Kembali ke akomodasi, ambil barang, dan menuju Bandara Internasional Ngurah Rai (DPS) untuk keberangkatan.
    *   **Perkiraan Biaya:** Transportasi ke bandara (Rp 100.000 - Rp 150.000).

*   **Perkiraan Biaya Total Hari 3:** Rp 300.000 - Rp 650.000 (tergantung pilihan destinasi).

---

**Perkiraan Total Biaya Perjalanan (3 Hari, 2 Malam):**

*   **Akomodasi (2 malam, standar):** Rp 500.000 - Rp 800.000
*   **Transportasi Lokal (termasuk antar-jemput bandara, sewa motor di Nusa Penida):** Rp 700.000 - Rp 1.000.000
*   **Aktivitas & Tiket Masuk (termasuk Tari Kecak, Fast Boat):** Rp 750.000 - Rp 1.100.000
*   **Makan & Minum (3 hari):** Rp 500.000 - Rp 750.000

**Total Perkiraan Biaya Keseluruhan:** Rp 2.450.000 - Rp 3.650.000 per orang.

**Catatan Penting:**

*   **Akomodasi:** Pilihlah area Kuta, Seminyak, Legian untuk akses mudah ke pantai dan hiburan. Jika mencari ketenangan, pertimbangkan area Jimbaran atau Ungasan.
*   **Transportasi:** Menggunakan taksi online (GoCar/GrabCar) atau menyewa motor/mobil dengan sopir adalah pilihan terbaik. Untuk keliling Nusa Penida, sewa motor atau mobil dengan sopir direkomendasikan.
*   **Fleksibilitas:** Itinerary ini bersifat fleksibel. Anda bisa menyesuaikannya dengan minat Anda, misalnya menambah waktu untuk selancar atau *snorkeling*.
*   **Pemesanan:** Tiket *fast boat* ke Nusa Penida dan akomodasi sebaiknya dipesan jauh-jauh hari, terutama jika bepergian di musim ramai.
*   **Biaya:** Biaya ini adalah perkiraan dan dapat bervariasi tergantung pilihan Anda, tempat makan, dan gaya perjalanan.

Semoga itinerary ini membantu Anda merencanakan liburan pantai yang luar biasa di Bali! Selamat menikmati!

### Section 3: Try the ADK Web Interface¶

Overview
ADK includes a built-in web interface for interactively chatting with, testing, and debugging your agents.

In [15]:
!adk create sample-agent --model gemini-2.5-flash-lite --api_key $GOOGLE_API_KEY


Agent created in /Users/tarisyaqurrota/Documents/Learning IT Skill/GitHub/travel-itenerary-agent/sample-agent:
- .env
- __init__.py
- agent.py



In [ ]:
!adk web # Disarankan menjalankan ini di terminal terpisah jika di ipynb, notebook akan terblokir